In [1]:
import pychor
import galois
import pandas as pd
import numpy as np
from dataclasses import dataclass

p = 2**31-1
GF = galois.GF(p)
p1 = pychor.Party('p1')
p2 = pychor.Party('p2')
dealer = pychor.Party('dealer')

@pychor.local_function
def share(x):
    s1 = GF.Random()
    s2 = GF(x) - s1
    return s1, s2

def deal_shares(x):
    s1, s2 = share(x).untup(2)
    s1.send(dealer, p1)
    s2.send(dealer, p2)
    return s1, s2

def deal_triple():
    # Step 1: generate a, b, c
    a = dealer.constant(GF.Random())
    b = dealer.constant(GF.Random())
    c = a * b

    # Step 2: secret share a, b, c
    a1, a2 = deal_shares(a)
    b1, b2 = deal_shares(b)
    c1, c2 = deal_shares(c)
    return (a1, a2), (b1, b2), (c1, c2)

def protocol_mult(x, y, triple):
    x1, x2 = x
    y1, y2 = y
    (a1, a2), (b1, b2), (c1, c2) = triple

    # Step 1. P1 computes d_1 = x_1 - a_1 and sends the result to P2
    d1 = x1 - a1
    d1.send(p1, p2)

    # Step 2. P2 computes d_2 = x_2 - a_2 and sends the result to P1
    d2 = x2 - a2
    d2.send(p2, p1)

    # Step 3: P1 and P2 both compute $d = d_1 + d_2 = x - a$
    d = d1 + d2

    # Step 4. P1 computes e_1 = y_1 - b_1 and sends the result to P2
    e1 = y1 - b1
    e1.send(p1, p2)

    # Step 5. P2 computes e_2 = y_2 - b_2 and sends the result to P1
    e2 = y2 - b2
    e2.send(p2, p1)

    # Step 6. P1 and P2 both compute $e = e_1 + e_2 = y - b$
    e = e1 + e2

    # Step 7. P1 computes r_1 = d*e + d*b_1 + e*a_1 + c_1
    r_1 = d * e + d * b1 + e * a1 + c1

    # Step 8. P2 computes r_2 = 0 + d*b_2 + e*a_2 + c_2
    r_2 = d * b2 + e * a2 + c2

    return r_1, r_2


# Building Applications with MPC

We saw a very simple application of MPC in Chapter 2 (counting the number of patients with heart disease, without sharing the underlying data), and we've now seen protocols for doing both addition and multiplication. What kinds of applications can we build with these tools, and how can we design APIs that make it easy to do?

Throughout this book, we'll use a style of encapsulating secret-shared data with *special objects* and defining operations on those objects using the fundamental MPC protocols we've learned about. For example, we'll start this chapter by defining a `SecInt` class for holding secret-shared integers, and we'll define addition and multiplication for these objects using the additive homomorphism and multiplication triples, respectively. This approach enables building applications that look very similar to traditional centralized Python programs, so that we don't need to worry about the details of the protocol when building applications.

An alternative style, used commonly in the MPC literature, is to define MPC protocols that process *circuits*, and then to build applications by defining circuits. The circuits used in MPC are similar to hardware circuits, but have only addition and multiplication gates, and each wire in the circuit represents a secret-shared value. We'll cover circuits in detail in Chapter TODO.

## `SecInt`: Secure Integers

We define the `SecInt` class as an abstract representation of secret-shared integers. We'll use Python's [`dataclass`](https://docs.python.org/3/library/dataclasses.html) decorator, since `SecInt` is a container for secret-shared data; the class will have two fields `s1` and `s2` to hold the additive secret shares defining the represented integer. We'll also define several methods:

- The `__add__` method will add two `SecInt` objects by adding the respective secret shares
- The `__mul__` method will multiply two `SecInt` objects using the `protocol_mult` protocol defined in Chapter 4
- The `reveal` method will reveal the value of the `SecInt` object by broadcasting the shares and reconstructing the secret
- The `input` class method will construct at new `SecInt` object by secret sharing the input value

The definition of the class appears below.

In [2]:
@dataclass
class SecInt:
    # s1 is p1's share of the value, and s2 is p2's share
    s1: galois.GF
    s2: galois.GF

    @classmethod
    def input(cls, val):
        """Secret share an input: p1 holds s1, and p2 holds s2"""
        s1, s2 = share(val).untup(2)
        if p1 in val.parties:
            s2.send(p1, p2)
            return SecInt(s1, s2)
        else:
            s1.send(p2, p1)
            return SecInt(s1, s2)

    def __add__(x, y):
        """Add two SecInt objects using local addition of shares"""
        return SecInt(x.s1 + y.s1, x.s2 + y.s2)

    def __mul__(x, y):
        """Multiply two SecInt objects using a triple"""
        triple = multiplication_triples.pop()
        r1, r2 = protocol_mult((x.s1, x.s2), (y.s1, y.s2), triple)
        return SecInt(r1, r2)

    def reveal(self):
        """Reveal the secret value by broadcast and reconstruction"""
        self.s1.send(p1, p2)
        self.s2.send(p2, p1)
        return self.s1 + self.s2

Now we can write choreographic programs describing protocols that operate on `SecInt` objects as if they're regular Python numbers. For example, below a program that computes $(x+y)^3$ without revealing $x$ or $y$. Note that every multiplication of `SecInt` objects consumes a multiplication triple, so our program needs to begin by populating the global pool of multiplication triples as a preprocessing step.

In [136]:
multiplication_triples = []

with pychor.LocalBackend():
    x_input = p1.constant(3)
    y_input = p2.constant(4)

    # Preprocessing: deal multiplication triples
    for _ in range(200):
        multiplication_triples.append(deal_triple())

    # Create secret shares of the inputs
    x = SecInt.input(x_input)
    y = SecInt.input(y_input)

    # Online phase: compute (x+y)^3
    z = x + y
    result = z*z*z
    print('(x+y)^3:', result.reveal())

(x+y)^3: 343@{p2, p1}


## Application: Cross-Tabulation


One common analysis of datasets that involves only addition of integers is *[cross tabulation](https://en.wikipedia.org/wiki/Contingency_table)*, the process of counting the number of people in the dataset with particular combinations of attributes. In our heart disease example, we might want to answer a question like: "do people who experience exercise-induced angina more commonly have heart disease?" To answer this question, we can count how many people have:

- Neither exercise-induced angina nor heart disease
- Exercise-induced angina, but not heart disease
- No exercise-induced angina, but heart disease
- Both exercise-induced angina and heart disease

The results will allow comparing the groups to determine the link between the two.

In our framework, the easiest way to do this is to use the [Pandas function for cross tabulation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) and then convert the values to a tuple. This approach allows us to separate the values in the tuple and secret share each one of them. The following function builds the contingency table locally, using a single dataset:

In [4]:
@pychor.local_function
def heart_disease_crosstab(df):
    return tuple(pd.crosstab(df['exang'], df['target']).to_numpy().flatten())

Now we can write a program to read in separate datasets for P1 and P2, secret share the associated contingency tables, and add corresponding values:

In [5]:
with pychor.LocalBackend():
    # Read datasets
    df1 = pychor.locally(pd.read_csv, 'heart1.csv'@p1)
    df2 = pychor.locally(pd.read_csv, 'heart2.csv'@p2)

    # Create crosstabs
    crosstab1 = heart_disease_crosstab(df1).untup(4)
    crosstab2 = heart_disease_crosstab(df2).untup(4)

    # Secret share crosstabs
    sec_crosstab1 = [SecInt.input(v) for v in crosstab1]
    sec_crosstab2 = [SecInt.input(v) for v in crosstab2]

    # Add corresponding values and reveal results
    crosstab = [(x + y).reveal() for x, y in zip(sec_crosstab1, sec_crosstab2)]
    crosstab_np = np.array(crosstab).reshape((2,2))
    print('Final crosstab:')
    print(crosstab_np)

Final crosstab:
[[39@{p2, p1} 95@{p2, p1}]
 [49@{p2, p1} 17@{p2, p1}]]


The results show that rates of exercise-induced angina are not too far apart for patients without heart disease, but there's a significant difference between them for patients with heart disease! Many useful analyses can be performed this way, with only addition of data.

## `SecSignedInt`: Signed Integers

So far we've only dealt with non-negative integers. As long as they're smaller than the characteristic of the finite field we use, non-negative integers can be directly encoded as field elements. If we want to encode *negative* numbers, things get more complicated, since finite fields don't have negative elements.

The most common approach in MPC protocols is actually quite similar to the encoding of signed integers in binary. To encode a signed number $n$ as a field element in $GF(p)$, we compute $n \mod p$:

In [18]:
def encode_signed(n):
    return GF(n % p)

This has the effect of encoding non-negative numbers as the equivalent field element:

In [19]:
encode_signed(1)

GF(1, order=2147483647)

It has an interesting impact on negative numbers: a negative number $n$, encoded as $n \mod p$, becomes $p - \lvert n \rvert$:

In [20]:
encode_signed(-1)

GF(2147483646, order=2147483647)

A big advantage of this encoding is that field arithmetic basically translates into signed arithmetic on the encoded numbers. For example, $3 - 1$ should be equal to $2$, and it is:

In [21]:
encode_signed(3) + encode_signed(-1)

GF(2, order=2147483647)

What about *decoding* these numbers, back to signed integers? We'll treat the first $p/2$ field elements as non-negative numbers, and the remaining ones as negative numbers. Decoding the non-negative numbers doesn't change their values. We decode negative numbers by subtracting $p$, so that when composed with our encoding procedure we get $p - \lvert n \rvert - p = -\lvert n \rvert$.

In [22]:
def decode_signed(x):
    if x < p/2:
        return int(x)
    else:
        return int(x) - p

print(f'3 encoded = {encode_signed(3)}; decoded = {decode_signed(encode_signed(3))}')
print(f'-3 encoded = {encode_signed(-3)}; decoded = {decode_signed(encode_signed(-3))}')

3 encoded = 3; decoded = 3
-3 encoded = 2147483644; decoded = -3


We've now effectively divided our finite field in half, so we've halved the maximum value we can represent without overflow (and we want to avoid overflow, since it is likely to cause the wrong answer). For example, if we multiply two large positive numbers, we can easily end up with a negative one:

In [23]:
decode_signed(encode_signed(2**15) * encode_signed(2**15))

-1073741823

This is a problem with any binary number encoding a signed integer, and it [even causes problems in NumPy](https://stackoverflow.com/questions/1970680/integer-overflow-in-numpy-arrays). The solution is just to be careful about the size of the finite field you use for your application, to ensure that overflow isn't a problem.

We can define a class called `SecSignedInt` to represent signed integers using the encoding and decoding procedures we've defined, building on the `SecInt` class. The only differences are in sharing (we encode the number before sharing) and revealing (we decode the number before returning it).

In [30]:
@pychor.local_function
def encode_signed(n):
    return GF(n % p)

@pychor.local_function
def decode_signed(x):
    if x < p/2:
        return int(x)
    else:
        return int(x) - p

@dataclass
class SecSignedInt:
    # s1 is p1's share of the value, and s2 is p2's share
    s1: galois.GF
    s2: galois.GF

    @classmethod
    def input(cls, val):
        """Secret share an input: p1 holds s1, and p2 holds s2"""
        encoded_val = encode_signed(val)
        s1, s2 = share(encoded_val).untup(2)
        if p1 in val.parties:
            s2.send(p1, p2)
            return SecSignedInt(s1, s2)
        else:
            s1.send(p2, p1)
            return SecSignedInt(s1, s2)

    def __add__(x, y):
        """Add two SecInt objects using local addition of shares"""
        return SecSignedInt(x.s1 + y.s1, x.s2 + y.s2)

    def __mul__(x, y):
        """Multiply two SecInt objects using a triple"""
        triple = multiplication_triples.pop()
        r1, r2 = protocol_mult((x.s1, x.s2), (y.s1, y.s2), triple)
        return SecSignedInt(r1, r2)

    def reveal(self):
        """Reveal the secret value by broadcast and reconstruction"""
        self.s1.send(p1, p2)
        self.s2.send(p2, p1)
        return decode_signed(self.s1 + self.s2)

Here's a simple example of adding a positive and negative number, resulting in a negative number:

In [31]:
with pychor.LocalBackend():
    x = SecSignedInt.input(p1.constant(3))
    y = SecSignedInt.input(p2.constant(-5))
    print('Result:', (x+y).reveal())

Result: -2@{p2, p1}


## `SecDec`: Decimal Numbers

What if we want to represent decimal numbers? Finite fields definitely can't do that! The most common approaches in MPC are inspired by techniques used in binary circuits used to build computers, such as [fixed-point arithmetic](https://en.wikipedia.org/wiki/Fixed-point_arithmetic) and [floating-point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic). However, the MPC versions of these techniques usually differ a bit from the hardware versions, because some things that are easy in a binary circuit are much more difficult using operations over finite fields.

The basic idea behind fixed- or floating-point numbers is that we can represent a decimal number like $0.3$ as an integer multiplied by some power of 10: $0.3 = 3 * 10^{-1}$. Computer hardware usually uses a base of 2 (instead of 2) because multiplying by a power of 2 can be accomplished via shifting. In the MPC world, we can use any base; we'll use 10 in this section for readability.

We can use this technique to design a function for encoding a decimal number as a finite field element:

In [38]:
def encode_decimal(n, power=2):
    encoded_int = int(n * 10**power)
    assert abs(encoded_int) < p/2  # avoid overflow in the encoding
    return GF(encoded_int % p)

In [39]:
encode_decimal(.3)

GF(30, order=2147483647)

In [40]:
encode_decimal(35.63)

GF(3563, order=2147483647)

In [41]:
# We lose precision if the power is not large enough
encode_decimal(0.12345)

GF(12, order=2147483647)

In [42]:
# We regain precision by increasing the power
encode_decimal(0.12345, power=5)

GF(12345, order=2147483647)

Decoding is the inverse: we convert to an integer, then divide by the value we multiplied by during encoding. We incorporate the technique used above for signed numbers in our decoding procedure, so that we can represent negative decimals too.

In [43]:
def decode_decimal(x, power=2):
    if x < p/2:
        int_encoded = int(x)
    else:
        int_encoded = int(x) - p
    return int_encoded / 10**power

In [44]:
decode_decimal(encode_decimal(.3))

0.3

In [45]:
decode_decimal(encode_decimal(-.3))

-0.3

This is a useful representation because the operations of finite field arithmetic translate (with some caveats) into equivalent operations on the encoded decimal numbers. For example, we can add encoded decimals and we get the right answer:

In [59]:
x = encode_decimal(0.3)
y = encode_decimal(34.6)
z = x + y
print('Encoded x:', x, 'encoded y:', y, 'encoded z:', z)
print('z decoded:', decode_decimal(z))

Encoded x: 30 encoded y: 3460 encoded z: 3490
z decoded: 34.9


Things are more complicated for multiplication, because $(x * 10^2)(y * 10^2) = xy * 10^4$: *the power goes up!* This is not necessarily a problem, as long as we decode the resulting number correctly to account for the increase in power:

In [63]:
x = encode_decimal(0.3)
y = encode_decimal(34.6)
z = x * y
print('Encoded x:', x, 'encoded y:', y, 'encoded z:', z)
print('z decoded (wrong power):', decode_decimal(z))
print('z decoded (right power):', decode_decimal(z, power=4))

Encoded x: 30 encoded y: 3460 encoded z: 103800
z decoded (wrong power): 1038.0
z decoded (right power): 10.38


There is one problem with this solution: each multiplication significantly increases the size of the encoded number because of the presence of the scaling factor, and this quickly leads to overflow. With our example numbers, $x * y * y * y$ is represented as $x*y*y*y*10^8$, which is already larger than $p/2$ and leads to overflow:

In [83]:
x = encode_decimal(0.3)
y = encode_decimal(34.6)
z = x * y * y * y
print('Encoded x:', x, 'encoded y:', y, 'encoded z:', z)
print('z decoded (right power; wrong answer):', decode_decimal(z, power=8))

Encoded x: 30 encoded y: 3460 encoded z: 1406532034
z decoded (right power; wrong answer): -7.40951613


Fixed-point arithmetic implementations typically deal with this issue by *scaling* the encoded numbers back down after multiplying (e.g. by dividing by $10^2$), causing a loss of precision but avoiding overflow. Floating-point arithmetic uses a similar process called *normalization*. In hardware, when the base is 2, this is easy to do with an arithmetic shift; in MPC, division (no matter the base) is not so easy.

For now, we'll ignore the problem, and return to it later and implement a (relatively complicated) solution called *truncation*.

We can define a class called `SecDec` for decimal numbers in an analagous way to the previous classes. We represent the current power of a decimal number explicitly in the object, as a *public* value.

In [103]:
@pychor.local_function
def encode_decimal(n, power=2):
    encoded_int = int(n * 10**power)
    assert abs(encoded_int) < p/2  # avoid overflow in the encoding
    return GF(encoded_int % p)

@pychor.local_function
def decode_decimal(x, power=2):
    if x < p/2:
        int_encoded = int(x)
    else:
        int_encoded = int(x) - p
    return int_encoded / 10**power

@dataclass
class SecDec:
    s1: galois.GF
    s2: galois.GF
    power: int

    @classmethod
    def input(cls, val, power=2):
        encoded_val = encode_decimal(val, power)
        s1, s2 = share(encoded_val).untup(2)
        if p1 in val.parties:
            s2.send(p1, p2)
            return SecDec(s1, s2, power)
        else:
            s1.send(p2, p1)
            return SecDec(s1, s2, power)

    def __add__(x, y):
        assert x.power == y.power
        return SecDec(x.s1 + y.s1, x.s2 + y.s2, x.power)

    def __mul__(x, y):
        triple = multiplication_triples.pop()
        r1, r2 = protocol_mult((x.s1, x.s2), (y.s1, y.s2), triple)
        return SecDec(r1, r2, x.power + y.power)

    def reveal(self):
        self.s1.send(p1, p2)
        self.s2.send(p2, p1)
        return decode_decimal(self.s1 + self.s2, self.power)

In [106]:
with pychor.LocalBackend():
    x = SecDec.input(p1.constant(0.4))
    y = SecDec.input(p2.constant(-5.5))
    print('x + y:', (x+y).reveal())
    print('x * y:', (x*y).reveal())
    print('x * y * y:', (x*y*y).reveal())

x + y: -5.1@{p2, p1}
x * y: -2.2@{p2, p1}
x * y * y: 12.1@{p2, p1}


This approach is not quite the same as either floating-point or fixed-point arithmetic - it's not fixed-point, since the power varies as operations are performed, and it's not floating-point, since it lacks normalization and the power is public.

## Division

Now that we have decimal numbers, can we perform division?

The answer is "kind of." Division is just multiplication by the reciprocal ($x/y = x * (1/y)$), so if we have a secret-shared reciprocal (or if the denominator is public, or known to one of the parties) then our `SecDec` class can already do the operation. Here's an example where we compute $34 / 3$:

In [110]:
with pychor.LocalBackend():
    x = SecDec.input(p1.constant(34))
    y = SecDec.input(p2.constant(.33))
    print('34 / 3:', (x*y).reveal())

34 / 3: 11.22@{p2, p1}


But, what if the denominator is unknown to all parties? In that case, we need to *compute* its reciprocal using our MPC operations, which we can't do directly.

One common approach is to *approximate* the reciprocal using [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method#Multiplicative_inverses_of_numbers_and_power_series), which only requires multiplication and subtraction (of decimal numbers). This approach involves an iterative algorithm that starts from a guess for the reciprocal and converges to the actual reciprocal (as long as the guess is decent). It usually produces a decent approximation after just a few iterations.

In [131]:
def reciprocal(x, x_reciprocal_guess):
    x_reciprocal = x_reciprocal_guess
    for i in range(5):
        print(f'Before iteration {i}, the reciprocal is {x_reciprocal}')
        x_reciprocal = (2 - x * x_reciprocal) * x_reciprocal
    return x_reciprocal

In [132]:
reciprocal(3, .1)

Before iteration 0, the reciprocal is 0.1
Before iteration 1, the reciprocal is 0.17
Before iteration 2, the reciprocal is 0.2533
Before iteration 3, the reciprocal is 0.31411733000000003
Before iteration 4, the reciprocal is 0.3322255689810133


0.3333296519077525

We can combine this idea with the `SecDec` class to build a secure reciprocal protocol. We will need to be careful about how many iterations we perform, however, since our approach overflows quickly as the number of multiplications increases, and each iteration of the algorithm performs 3 multiplications. We perform subtraction by multiplying the subtrahend by -1 and then adding.

In [146]:
def protocol_reciprocal(x, x_reciprocal_guess):
    neg_one = SecDec.input(p1.constant(-1), power=0)
    x_reciprocal = x_reciprocal_guess
    for i in range(3):
        prod1 = neg_one * x * x_reciprocal
        x_reciprocal = (prod1 + SecDec.input(p1.constant(2), power=prod1.power)) * x_reciprocal
    return x_reciprocal

In [149]:
with pychor.LocalBackend():
    x = SecDec.input(p1.constant(34))
    y = SecDec.input(p1.constant(3), power=0)
    guess = SecDec.input(p1.constant(.2), power=1)
    y1 = protocol_reciprocal(y, guess)
    print('1/3:', y1.reveal())
    print('34 / 3:', (x*y1).reveal())

1/3: 0.33311488@{p2, p1}
34 / 3: -0.0557574091@{p2, p1}


With 3 iterations, a good guess, and being careful with the starting powers for our encodings (e.g. using `power=0` when we encode integers, to avoid increasing the power unnecessarily), we're able to calculate a pretty good approximation of the reciprocal without overflowing. However, we get the wrong answer when multiplying by the reciprocal to perform division, due to overflow. We can fix the problem in this case by using a larger field to avoid overflow:

In [152]:
p = 2**61-1
GF = galois.GF(p)
multiplication_triples = []

with pychor.LocalBackend():
    # Deal new multiplication triples in the new field
    for _ in range(200):
        multiplication_triples.append(deal_triple())

    x = SecDec.input(p1.constant(34))
    y = SecDec.input(p1.constant(3), power=0)
    guess = SecDec.input(p1.constant(.2), power=1)
    y1 = protocol_reciprocal(y, guess)
    print('1/3:', y1.reveal())
    print('34 / 3:', (x*y1).reveal())

1/3: 0.33311488@{p2, p1}
34 / 3: 11.32590592@{p2, p1}


This approach is clearly limited, since increasing the field size has significant drawbacks (e.g. memory usage and computation time for field operations). However, for many applications, a few multiplications is enough, and even limited division is sufficient to get good results.

## Application: Computing the Average Age of Heart Disease Patients

We now have the tools needed to build a protocol that computes the average age of heart disease patients in the heart disease dataset, without revealing the sum of ages or the count of patients. We'll use the `SecDec` class to represent numbers, and the approach described above for computing the reciprocal. The guess for the reciprocal is important for ensuring we get the right answer, and this can be challenging in many applications. In this one, however, it's easy: each party's data has about half of the heart disease patients, so doubling the number of heart disease patients a single party has is a decent estimate of the total, and can be used to make a good guess at the reciprocal. Since the reciprocal is smaller in this example than the last one, we'll need to use an even larger finite field to avoid overflow. We include some debugging printouts to show the protocol working, but in a real deployment we'd keep the reciprocal secret.

In [184]:
@pychor.local_function
def sum_age_heart_disease_patients(df):
    return df[df['target'] == 1]['age'].sum()

@pychor.local_function
def count_heart_disease_patients(df):
    return len(df[df['target'] == 1])

p = 2**127-1
GF = galois.GF(p)
multiplication_triples = []

with pychor.LocalBackend():
    # Deal new multiplication triples in the new field
    for _ in range(20):
        multiplication_triples.append(deal_triple())

    # Read datasets
    df1 = pychor.locally(pd.read_csv, 'heart1.csv'@p1)
    df2 = pychor.locally(pd.read_csv, 'heart2.csv'@p2)

    # Numerator: compute the total sum of all ages of heart disease patients
    sum1 = sum_age_heart_disease_patients(df1)
    sum2 = sum_age_heart_disease_patients(df2)

    total_sum = SecDec.input(sum1, power=0) + SecDec.input(sum2, power=0)

    # Denominator: compute the total number of heart disease patients
    count1 = count_heart_disease_patients(df1)
    count2 = count_heart_disease_patients(df2)

    total_count = SecDec.input(count1, power=0) + SecDec.input(count2, power=0)

    # Compute the reciprocal
    guess = 1 / (count1 * 2)
    print('Reciprocal guess:', guess)
    reciprocal = protocol_reciprocal(total_count, SecDec.input(guess, power=3))
    print('Secret-shared reciprocal:', reciprocal.reveal())
    print('Actual reciprocal:', 1/total_count.reveal())

    # Compute the average: numerator * reciprocal
    average = total_sum * reciprocal
    print('Final average:', average.reveal())

Reciprocal guess: 0.00909090909090909@{p1}
Secret-shared reciprocal: 0.008928571428571428@{p2, p1}
Actual reciprocal: 0.008928571428571428@{p2, p1}
Final average: 52.25@{p2, p1}
